Compil ou Face
========

Notions de compilation pour le reverseur
--------------------------------------------

par *Serge Guelton*☘ et *Adrien Guinet*⚽

☘ ``sguelton@quarkslab.com``

⚽ ``aguinet@quarkslab.com``

``man sguelton``
=========

- Ingénieur R&D à Quarkslab, spécialisé en compilation (Python, LLVM)
- Chercheur associé à Télécom Bretagne

``man aguinet``
=========

- Ingénieur R&D à Quarkslab, spécialisé en tout

Ce Cours (il est long)
============

- Connaitre sa chaîne de compilation
- Connaître son compilo
- Comprendre quelques transformations et analyse

Cours et TP entrelacés

Intérêt pour le reverser
=============

- Mieux comprendre le code généré
- Billes de compréhension pour écrire des outils d'analyse
- Culture G

Chaîne de compilation classique
=================

```

a.c -- a.o --
             :-- a.out 
b.c -- b.o --
```

In [1]:
%%file a.c
#include <stdio.h>
void greet(char const* who) {
    printf("Hello %s!\n", who);
}

Writing a.c


In [2]:
%%file b.c
extern void greet(char const* who);
int main(int argc, char const* argv[]) {
    if(argc == 1) greet("world");
    else greet(argv[1]);
    return 0;
}

Writing b.c


In [3]:
%%!
clang-3.7 a.c b.c
./a.out $USER

['Hello sguelton!']

Pas à Pas : Préprocesseur
==============

*a.k.a.* « le sed du pauvre »

In [4]:
!clang-3.7 -E a.c | wc -l

730


In [5]:
!clang-3.7 -E a.c | head -n 10

# 1 "a.c"
# 1 "<built-in>" 1
# 1 "<built-in>" 3
# 316 "<built-in>" 3
# 1 "<command line>" 1
# 1 "<built-in>" 2
# 1 "a.c" 2
# 1 "/usr/include/stdio.h" 1 3 4
# 27 "/usr/include/stdio.h" 3 4
# 1 "/usr/include/features.h" 1 3 4


Pas à Pas : La Représentation Interne
=====================

Généralement pas exposée à l'utilisateur…

In [6]:
!clang-3.7 -S -emit-llvm a.c
!head -n 16 a.ll

; ModuleID = 'a.c'
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

@.str = private unnamed_addr constant [11 x i8] c"Hello %s!\0A\00", align 1

; Function Attrs: nounwind uwtable
define void @greet(i8* %who) #0 {
  %1 = alloca i8*, align 8
  store i8* %who, i8** %1, align 8
  %2 = load i8*, i8** %1, align 8
  %3 = call i32 (i8*, ...) @printf(i8* getelementptr inbounds ([11 x i8], [11 x i8]* @.str, i32 0, i32 0), i8* %2)
  ret void
}

declare i32 @printf(i8*, ...) #1


Pas à Pas : La transformation de RI (olé !)
====================

Ce qui se cache derrière ``-O2`` et consorts

In [7]:
!opt-3.7 -mem2reg a.ll -S | head -n 10

; ModuleID = 'a.ll'
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

@.str = private unnamed_addr constant [11 x i8] c"Hello %s!\0A\00", align 1

; Function Attrs: nounwind uwtable
define void @greet(i8* %who) #0 {
  %1 = call i32 (i8*, ...) @printf(i8* getelementptr inbounds ([11 x i8], [11 x i8]* @.str, i32 0, i32 0), i8* %who)
  ret void


Pas à Pas : La génération de code assembleur
========================

Avec la bonne variante syntaxique…

In [8]:
!llc-3.7 a.ll --x86-asm-syntax=intel
!head -n 23 a.s | tail -n 10

	mov	rbp, rsp
.Ltmp2:
	.cfi_def_cfa_register rbp
	sub	rsp, 16
	mov	rcx, rdi
	mov	qword ptr [rbp - 8], rcx
	mov	edi, .L.str
	xor	eax, eax
	mov	rsi, rcx
	call	printf


Ou plus simplement :



In [9]:
!clang-3.7 -S  a.c #-masm=intel if needed

Pas à Pas : L'assemblage
==============

Génération de code objet, ou ``.o``

In [10]:
!as a.s -o a.o
!file a.o

a.o: ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), not stripped


Pas à Pas : L'édition de liens
=================

Le *linker*, les libs statiques, les lib dynamiques…

In [11]:
!clang-3.7 -c b.c # pour avoir le deuxième code objet

In [12]:
!nm b.o | grep greet

                 U greet


In [13]:
!nm a.o | grep greet
!nm a.o | grep printf

0000000000000000 T greet
                 U printf


In [14]:
!ldd /bin/ls | grep libc.so

	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f447e1ad000)


In [15]:
!nm /lib/x86_64-linux-gnu/libc.so.6 | grep printf

nm: /lib/x86_64-linux-gnu/libc.so.6: no symbols


In [16]:
!file -L /lib/x86_64-linux-gnu/libc.so.6

/lib/x86_64-linux-gnu/libc.so.6: ELF 64-bit LSB shared object, x86-64, version 1 (GNU/Linux), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, BuildID[sha1]=4c15ef4771b6f2882c724245afbe03b677466ac5, for GNU/Linux 2.6.32, stripped


In [17]:
!readelf -s /lib/x86_64-linux-gnu/libc.so.6 | grep ' printf'

   596: 0000000000050d60   161 FUNC    GLOBAL DEFAULT   12 printf@@GLIBC_2.2.5
  1482: 0000000000050cb0    31 FUNC    GLOBAL DEFAULT   12 printf_size_info@@GLIBC_2.2.5
  1890: 0000000000050480  2086 FUNC    GLOBAL DEFAULT   12 printf_size@@GLIBC_2.2.5


Pas à Pas : l'exécutable
==============

Pour pondre un joli petit ``a.out`` tout mignon

In [18]:
! ld a.o b.o && ./a.out

/usr/bin/x86_64-linux-gnu-ld.bfd.real: warning: cannot find entry symbol _start; defaulting to 00000000004000b0
a.o: In function `greet':
a.c:(.text+0x20): undefined reference to `printf'


In [19]:
! ld a.o b.o -lc && file a.out && ./a.out

/usr/bin/x86_64-linux-gnu-ld.bfd.real: warning: cannot find entry symbol _start; defaulting to 0000000000400270
a.out: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib/ld64.so.1, not stripped
/bin/sh: 1: ./a.out: not found


In [20]:
! clang-3.7 -v a.o b.o 2>&1 | grep ld

 "/usr/bin/ld" --hash-style=both --build-id --eh-frame-hdr -m elf_x86_64 -dynamic-linker /lib64/ld-linux-x86-64.so.2 -o a.out /usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/../../../x86_64-linux-gnu/crt1.o /usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/../../../x86_64-linux-gnu/crti.o /usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/crtbegin.o -L/usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1 -L/usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/../../../x86_64-linux-gnu -L/lib/x86_64-linux-gnu -L/lib/../lib64 -L/usr/lib/x86_64-linux-gnu -L/usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/../../.. -L/usr/lib/llvm-3.7/bin/../lib -L/lib -L/usr/lib a.o b.o -lgcc --as-needed -lgcc_s --no-as-needed -lc -lgcc --as-needed -lgcc_s --no-as-needed /usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/crtend.o /usr/bin/../lib/gcc/x86_64-linux-gnu/5.2.1/../../../x86_64-linux-gnu/crtn.o


In [21]:
! ./a.out

Hello world!


Comprendre l'*Abstract Syntax Tree*
===================

Jouons avec Python et son AST, plus facile que celui de C++

In [22]:
import ast
tree = ast.parse("print(1)")
print(tree)

In [23]:
ast.dump(tree)

"Module(body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Num(n=1)], keywords=[], starargs=None, kwargs=None))])"

In [24]:
import astdump
astdump.indented(tree)

Module
  Expr
    Call
      Name
        Load
      Num


Compilation de l'AST
===========

Passage d'une représentation proche du langage à une représentation proche de l'interpréteur.

In [25]:
code = compile(tree, '<>', 'exec')

In [26]:
eval(code)

1


Inspection du bytecode
=============

CPython → Interpréteur à pile

In [27]:
import dis
dis.dis(code)

  1           0 LOAD_NAME                0 (print)
              3 LOAD_CONST               0 (1)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP
             10 LOAD_CONST               1 (None)
             13 RETURN_VALUE


``(`` Aparté
========

D'après vous, quels sont les avantages et inconvénients d'une interpréteur à pile par rapport à un interpréteur à registre?

Interpréteur à pile
----------------------

Facile de conception, peu d'optimisations

Interpréteur à registre
--------------------------

Plus complexe (et pas seulement pour l'allocation de registre) mais permet de modéliser plus d'optimisations

Game of Stack
=======

Écrire un interpréteur (dans le langage de votre choix) qui comprend les instructions suivantes :

- ``PUSH <integer>`` qui ajoute ``<integer>`` au dessus de la pile
- ``DUP`` qui duplique le dessus de la pile
- ``ADD`` qui enlève les deux premiers éléments de la pile et ajoute ``S[0] + S[1]`` au dessus de la pile
- ``MUL`` qui enlève les deux premiers éléments de la pile et ajoute ``S[0] * S[1]`` au dessus de la pile
- ``READ`` qui lit un entier sur ``stdin`` et l'ajoute au dessus de la pile
- ``WRITE`` qui dépile le premier élément de la pile et l'affiche sudr ``stdout``


Par exemple :

```
0 READ
1 DUP
2 ADD
3 WRITE
```


Introduisons maintenant une optmisation (de ouf !). Les deux séquences suivantes sont équivalentes :

```
PUSH 2
MUL
```

et

```
DUP
ADD
```

Ajoutez à votre interpréteur une passe qui effectue de qui s'avère être une *peephole optimisation* en transformant l'une en l'autre.


Fin de l'aparté ``)``
==========

Continuons à jouer avec l'AST
================

L'AST Python peut être parcouru grâce à un **visiteur** (qui n'est pas né d'hier)

À lire : https://docs.python.org/3/library/ast.html

In [28]:
class VisitIntegers(ast.NodeVisitor):
    def visit_Num(self, node):
        if isinstance(node.n, int):
            print(node.n)

VisitIntegers().visit(tree)

1


Exo
===

Écrivez un visiteur qui va trouver tous les appels à la fonction ``open``

Pourquoi est-ce en fait impossible en analyse statique ?

En instrumentant
=========

Une sorte d'analyse dynamique ?

In [29]:
import builtins
real_open = builtins.open
def myopen(*args, **kwargs):
    print(args, kwargs)
    return real_open(*args, **kwargs)
builtins.open = myopen
open("/dev/null")

('/dev/null',) {}


<_io.TextIOWrapper name='/dev/null' mode='r' encoding='UTF-8'>

In [30]:
builtins.open = real_open

Comprendre la Représentation Interne (IR dans la langue de Homer)
=======================

En se basant sur celle de LLVM.

Étudions plus précisément la sortie de clang sur un code simple :

In [31]:
%%file mask.c
void mask(unsigned n, int data[n], int mask) {
    for(unsigned i = 0; i < n; ++i)
        data[i] ^= mask;
}

Writing mask.c


In [32]:
! clang-3.7 -O1 -S -emit-llvm -o - mask.c | sed -n '6,18 p'

define void @mask(i32 %n, i32* nocapture %data, i32 %mask) #0 {
  %1 = icmp eq i32 %n, 0
  br i1 %1, label %._crit_edge, label %.lr.ph

._crit_edge:                                      ; preds = %.lr.ph, %0
  ret void

.lr.ph:                                           ; preds = %0, %.lr.ph
  %indvars.iv = phi i64 [ %indvars.iv.next, %.lr.ph ], [ 0, %0 ]
  %2 = getelementptr inbounds i32, i32* %data, i64 %indvars.iv
  %3 = load i32, i32* %2, align 4, !tbaa !1
  %4 = xor i32 %3, %mask
  store i32 %4, i32* %2, align 4, !tbaa !1


Représentation Hiérarchique
===============

Un Module contient :
- des variables globales
- des fonctions
- des métadonnées

Représentation Hiérarchique
===============

Une fonction contient des blocs de base…

Qui contiennent des instructions…

Qui finissent par un…

Terminator
===============

- Ils terminent (d'oh) un bloc de base
- Transfèrent le flot de contrôle à un autre bloc (``br``) à l'appelant (``ret``), à un gestionnaire d'exception (``invoke``) ou à rien (``unreachable``

```
                   <((((((\\\
                   /      . }\
                   ;--..--._|}
(\                 '--/\--'  )
 \\                | '-'  :'|
  \\               . -==- .-|
   \\               \.__.'   \--._
   [\\          __.--|       //  _/'--.
   \ \\       .'-._ ('-----'/ __/      \
    \ \\     /   __>|      | '--.       |
     \ \\   |   \   |     /    /       /
      \ '\ /     \  |     |  _/       /
       \  \       \ |     | /        /
 snd    \  \      \        /
 ```
 
 source: http://www.ascii-code.com/ascii-art/movies/other.php

Les instructions
=========

- Produisent et utilisent des **valeurs** stockées dans des **registres virtuels**

```
%4 = xor i32 %3, %mask
```

- Manipulent des valeurs fortement typées :

```
%2 = getelementptr inbounds i32, i32* %data, i64 %indvars.iv
```

- Calcul, accès à la pile, appel de fonction, conversions…

- Peuvent être annotées (*metadata*)

```
%3 = load i32, i32* %2, align 4, !tbaa !1
...
!1 = !{!2, !2, i64 0}
!2 = !{!"int", !3, i64 0}
```

Forme *Single Static Assignment*
==================

- un registre virtuel ``%stuff`` n'est écrit qu'une fois
- la mémoire peut être lue ou écrite à travers ``load`` et ``store``
- l'instruction ``phi`` permet de faire un choix suivant le prédécesseur

```
%indvars.iv = phi i64 [ %indvars.iv.next, %.lr.ph ], [ 0, %0 ]
```

Avantages de cette réprésentation
================

- Propagation de constante qui rend cette situation impossible

```
%0 = i32 8
%1 = add i32 %param, %0
%2 = mul i32 %1, %0
```

- Élimination de code mort (bloc de base sans prédecesseur)

Et pleins de choses qui vont au delà de ce cours…

JIT do it
======

Les compilateurs à la volée permettent de :

- construire en mémoire une représentation du programme
- modifier cette représentation en fonction du contexte
- traduire cette représentation en code machine
- charger (incl. édition de lien) ce code dans l'espace mémoire du processus courant 


Exemple de JIT
==============

Le JIT de LLVM

In [33]:
!sed -n '/extern/,/BasicBloc/ p' interp.cpp

extern "C" int64_t Acc = 0;

void interp(char op, int64_t val) {
  llvm::LLVMContext& C = llvm::getGlobalContext();
  std::unique_ptr<llvm::Module> M{new llvm::Module {"", C}};
  auto* LT =  llvm::IntegerType::get(C, 64);
  auto* GV = new llvm::GlobalVariable(*M, LT, false, llvm::GlobalValue::ExternalLinkage, nullptr, "Acc");

  auto* FT = llvm::FunctionType::get(LT, false);
  auto* F = llvm::Function::Create(FT, llvm::GlobalValue::InternalLinkage, "runme", M.get());
  auto* BB = llvm::BasicBlock::Create(C, "", F, nullptr);


Exercice
=====

Étendez le code (téléchargeable sur https://gist.github.com/serge-sans-paille/aa332fa22692fcdfdc51) pour supporter plus d'opérations !

Comment le compilateur voit…
==================

Différents concepts de haut niveau passés à la moulinette

La vectorisation
==========

TL;DR : un registre vectoriel contient plusieurs scalaires et les opérations sur un registre vectoriel affectent chaque scalaire

```
[a0     [b0     [c0
 a1  +   b1  =   c1
 a2      b2      c2
 a3]     b3]     c3]
```

In [34]:
!clang-3.7 -O3 -march=native mask.c -S -emit-llvm -o - | sed -n '/vector.body:/,/xor <8/ p'

vector.body:                                      ; preds = %vector.body, %vector.ph
  %index = phi i64 [ 0, %vector.ph ], [ %index.next, %vector.body ]
  %5 = getelementptr inbounds i32, i32* %data, i64 %index
  %6 = bitcast i32* %5 to <8 x i32>*
  %wide.load = load <8 x i32>, <8 x i32>* %6, align 4, !tbaa !1
  %7 = getelementptr i32, i32* %5, i64 8
  %8 = bitcast i32* %7 to <8 x i32>*
  %wide.load5 = load <8 x i32>, <8 x i32>* %8, align 4, !tbaa !1
  %9 = getelementptr i32, i32* %5, i64 16
  %10 = bitcast i32* %9 to <8 x i32>*
  %wide.load6 = load <8 x i32>, <8 x i32>* %10, align 4, !tbaa !1
  %11 = getelementptr i32, i32* %5, i64 24
  %12 = bitcast i32* %11 to <8 x i32>*
  %wide.load7 = load <8 x i32>, <8 x i32>* %12, align 4, !tbaa !1
  %13 = xor <8 x i32> %wide.load, %broadcast.splat9


Une initialisation de structure
=================

In [35]:
%%file init_struct.c
struct foo { int a; float b; char c ;};
struct foo toir() {
    struct foo res = {.a = 0, .b =2.5, .c=3};
    return res;
}

Writing init_struct.c


In [36]:
!clang-3.7 -S -emit-llvm -O2 -o - init_struct.c | head -n 8

; ModuleID = 'init_struct.c'
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

; Function Attrs: nounwind readnone uwtable
define { i64, i8 } @toir() #0 {
  ret { i64, i8 } { i64 4620693217682128896, i8 3 }
}


Une classe avec méthode
=============

In [37]:
%%file class_with_method.cpp
class ik {
  public:
    int field;
    ik(int f) : field(f) {}
};
ik ariwarrior(ik const& ipe) {
    
    return ik(ipe.field);
}

Writing class_with_method.cpp


In [38]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep "%class.*type"

%class.ik = type { i32 }


In [39]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep "comdat"

$_ZN2ikC2Ei = comdat any
define linkonce_odr void @_ZN2ikC2Ei(%class.ik* nocapture %this, i32 %f) unnamed_addr #0 comdat align 2 {


In [40]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep this

define linkonce_odr void @_ZN2ikC2Ei(%class.ik* nocapture %this, i32 %f) unnamed_addr #0 comdat align 2 {
  %1 = getelementptr inbounds %class.ik, %class.ik* %this, i64 0, i32 0


In [41]:
!printf _ZN2ikC2Ei | c++filt

ik::ik(int)

Un ``memcpy``
========

In [42]:
%%file memcpy.c
#include <string.h>
void fast(size_t n, long *bacon, long const *antinople) {
    memcpy(bacon, antinople, sizeof(long) * n);
}

Writing memcpy.c


In [43]:
!clang-3.7 -S -emit-llvm memcpy.c -O2 -o - | sed -n '5,12 p'

; Function Attrs: nounwind uwtable
define void @fast(i64 %n, i64* nocapture %bacon, i64* nocapture readonly %antinople) #0 {
  %1 = bitcast i64* %bacon to i8*
  %2 = bitcast i64* %antinople to i8*
  %3 = shl i64 %n, 3
  tail call void @llvm.memcpy.p0i8.p0i8.i64(i8* %1, i8* %2, i64 %3, i32 8, i1 false)
  ret void
}


Note
====

Dans ``@llvm.memcpy.p0i8.p0i8.i64(i8* %1, i8* %2, i64 %3, i32 8, i1 false)``, le ``false`` indique un accès non volatile, koike ça vous évoque ?

Une levée d'exception
============


In [44]:
%%file raise.cpp
extern void foo();
void bar() {
    try { foo(); }
    catch(float) {}
}

Writing raise.cpp


In [45]:
!clang++-3.7 -S -emit-llvm -O2 -o - raise.cpp | sed -n '/invoke/,/resume/ p'

  invoke void @_Z3foov()
          to label %9 unwind label %1

; <label>:1                                       ; preds = %0
  %2 = landingpad { i8*, i32 }
          catch i8* bitcast (i8** @_ZTIf to i8*)
  %3 = extractvalue { i8*, i32 } %2, 1
  %4 = tail call i32 @llvm.eh.typeid.for(i8* bitcast (i8** @_ZTIf to i8*)) #3
  %5 = icmp eq i32 %3, %4
  br i1 %5, label %6, label %10

; <label>:6                                       ; preds = %1
  %7 = extractvalue { i8*, i32 } %2, 0
  %8 = tail call i8* @__cxa_begin_catch(i8* %7) #3
  tail call void @__cxa_end_catch() #3
  br label %9

; <label>:9                                       ; preds = %0, %6
  ret void

; <label>:10                                      ; preds = %1
  resume { i8*, i32 } %2
